In [2]:
import ee
import datetime
import os
import itertools
import sys
import re

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [3]:
ee.Initialize()

In [4]:
# append upper folder into sys-path during run time so we can
# import our own moduls
sys.path.append('..')

In [5]:
from BackGround_modules.Class_2_Classify_Fourier_Img   import Classification
from BackGround_modules.Class_3_Calculate_the_accuracy import Accuracy_assesment

##### Prepare basic parameters

In [6]:
# define the years to be classified
year_name  = list([f'{span[0]}_{span[1]}' for span in zip(range(1990,2020,3),range(1992,2020,3)) ])

# import north_china_plain boundary
North_China_Plain = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Boundary_shp/North_China_Plain_Boundary")

In [7]:
year_name

['1990_1992',
 '1993_1995',
 '1996_1998',
 '1999_2001',
 '2002_2004',
 '2005_2007',
 '2008_2010',
 '2011_2013',
 '2014_2016',
 '2017_2019']

##### Prepare input_image for classification

1) Climate date

In [22]:
# prepare the climate data
Climate_type = ['lrad','prec','pres','shum','srad','temp','wind']
Climate_mean = []

for year in year_name:   
    
    # define the three years according to year-variable
    year_start = int(year.split('_')[0])
    year_mid   = year_start + 1
    year_end   = year_start + 2
    
    # Initiate an empty list to hold the climate-tifs-mean from year_start to year_end
    climate_tifs_mean = []
    
    # compute the mean-tif of each climate-type
    for i in Climate_type:

        # Because cliate_data doesn't include data of 2019, so 
        # use if to exclede the data of year 2019
        if year_start < 2017:
            climate_tifs = [ee.Image(f"users/wangjinzhulala/North_China_Plain_Python/Climate_data/{i}/{i}_{t}")                        
                            for t in [year_start,year_mid,year_end]]
            tifs_mean    = ee.ImageCollection(climate_tifs).mean().toFloat().rename(i)

        else:
            climate_tifs = [ee.Image(f"users/wangjinzhulala/North_China_Plain_Python/Climate_data/{i}/{i}_{t}")                        
                            for t in [year_start,year_mid]]

            tifs_mean    = ee.ImageCollection(climate_tifs).mean().toFloat().rename(i)

        # add all climate_tifs_mean of year_start to year_end to list
        climate_tifs_mean.append(tifs_mean)
        
        climate_tifs_mean_img = ee.Image(climate_tifs_mean)
        

    # add the mean-cliamte-img into the Climate_mean
    Climate_mean.append(climate_tifs_mean_img)

2) Sentinel date

In [9]:
# Function to mask clouds using the Sentinel-2 QA band.
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask  = 1 << 10;
    cirrusBitMask = 1 << 11;

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    # Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask)\
                 .select("B.*")\
                 .copyProperties(image, ["system:time_start"])



# here composite Sentinel-2 multispectrum image of year [2015-2017] for classification of [2014-2016]
#            and Sentinel-2 multispectrum image of year [2015-2017] for classification of [2014-2016]
# because composition of S2 image at year [2014-2016] are not fully cloud-free
Sentinel_year_range = [("2018-01-01","2019-12-31"),("2015-01-01","2017-12-31")]

Sentinel_img =  [ee.ImageCollection("COPERNICUS/S2")\
                      .filterBounds(North_China_Plain)\
                      .filterDate(*S_t)\
                      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))\
                      .map(lambda x: maskS2clouds(x))\
                      .median() for S_t in Sentinel_year_range] 

# add 'Sentinel' to each band name of Sentinel img
Sentinel_img = [img.rename([f'Sentinel_{band}' for band in  img.bandNames().getInfo()]) for img in Sentinel_img]

3) Landsat date

In [17]:
# import Landsat img
Landsat_img = [ee.Image(f"users/wang8052664/Cloud_Free_Img/Landsat_cloud_free_{t}") for t in year_name]
# here add a 'Landsat' befor the name of each landsat band so to avoid duplicated names to Sentinel bands
Landsat_img = [img.rename([f'Landsat_{band}' for band in  img.bandNames().getInfo()]) for img in Landsat_img]

4) Fourier and Normalized-indexs date

In [26]:
# import Fourier img  
Fourier_img = [ee.Image(f"users/Jinzhu_Deakin/North_China_Plain/Fourier_img_harmonic_3/Fourier_img_{year}_harmonic_3")\
                 .clip(North_China_Plain)  for year in year_name[:5]] + \
              [ee.Image(f"users/wang8052664/North_China_Plain/Fourier_imgs/Fourier_img_{year}_harmonic_3")\
                 .clip(North_China_Plain)  for year in year_name[5:]] 

#Import the Index-mean Img
NDVI_img = [ee.Image(f"users/wensomone666/Jinzhu/Mean_NDVI/Year_{t}_Mean_NDVI").rename('Mean_NDVI') for t in year_name]
NDBI_img = [ee.Image(f"users/wensomone666/Jinzhu/Mean_NDVI/Year_{t}_Mean_NDBI").rename('Mean_NDBI') for t in year_name]
EVI_img  = [ee.Image(f"users/wensomone666/Jinzhu/Mean_NDVI/Year_{t}_Mean_EVI").rename('Mean_EVI')   for t in year_name]

5) DEM/Slope date

In [30]:
# Import DEM/SLOPE Img
DEM   = [ee.Image("USGS/SRTMGL1_003").rename('DEM')] * len(year_name)
SLOPE = [ee.Terrain.slope(DEM).rename('SLOPE')] * len(year_name)

In [32]:
# stack all Imput_Img together
Stack_img = [ee.Image(img) for img in zip(Landsat_img,Fourier_img,NDVI_img,NDBI_img,EVI_img,Climate_mean,DEM,SLOPE)]

##### Prepare input sample points

1) Extract image value to control plints

In [36]:
# Get the training sample
Control_points = ee.FeatureCollection('users/wangjinzhulala/North_China_Plain_Python/Sample_Points/Sentinel_2017_2019')

In [37]:
for t,img in zip(year_name,Stack_img):

    # define the gee-asset path for exporting
    path   = 'users/wangjinzhulala/North_China_Plain_Python/Sample_extract_img'
    name   = 'Re_inspect_sample_Landsat_ext_img'
    sample = Control_points

    Ext_sample = img.sampleRegions( collection = sample, 
                                    properties = ['Built'], 
                                    # here need to change scale to 10. If set as 30, the GEE will have to 
                                    # resample Sentinel img, which will leed to a 'Memory exceed problem' 
                                    scale      = 10, 
                                    geometries = True,
                                    tileScale  = 2)

    # exporting
    task = ee.batch.Export.table.toAsset(   collection  = Ext_sample,
                                            description = f'{name}_{t}',
                                            assetId     = f'{path}/{name}_{t}')
    task.start()


    # print out the process
    print(f'{name}_{t}')

Re_inspect_sample_Landsat_ext_img_1990_1992
Re_inspect_sample_Landsat_ext_img_1993_1995
Re_inspect_sample_Landsat_ext_img_1996_1998
Re_inspect_sample_Landsat_ext_img_1999_2001
Re_inspect_sample_Landsat_ext_img_2002_2004
Re_inspect_sample_Landsat_ext_img_2005_2007
Re_inspect_sample_Landsat_ext_img_2008_2010
Re_inspect_sample_Landsat_ext_img_2011_2013
Re_inspect_sample_Landsat_ext_img_2014_2016
Re_inspect_sample_Landsat_ext_img_2017_2019


2) fetch the control points with extracted img values

In [23]:
# Define the path to Sample_pt
path = 'users/wangjinzhulala/North_China_Plain_Python/Sample_extract_img'

# Get the training sample
Training_sample =[ee.FeatureCollection(f"{path}/Sentinel_Landsat_reinspect_sample_ext_img_{t}") for t in year_name]

#### Create the classification instances

In [24]:
# A two-layer loop to generate 10-randome-split layers for each year-period
Classificatioin_result = []

# The first-layer loop to go througth each year-periods
for t,in_img,pt in zip(year_name,Stack_img,Training_sample):
    
    classificaiton_tpye = 'Sentinel_Landsat'
    year                = t
    input_img           = in_img
    sample_pt           = pt
    input_band          = input_img.bandNames().getInfo()

    # The second-layer to go through each-randomnes
    for seed in range(10):

        # Instatiate the class with a name.
        classification = Classification(year_name      = year,
                                        Verified_point = sample_pt,
                                        Input_img      = input_img,
                                        Input_band     = input_band,
                                        Tree_num       = 100,
                                        seed           = seed,
                                        Zone_sample    = None,
                                        classProperty  = 'Built')


        # perform the classification on Input_img and Samples
        classification.Stp_1_Classification_on_img()
        classification.Stp_2_Classification_on_Samples()

        # Get the classified img
        classified_img = classification.classification_img

        # Get the classified samples
        train_sample_classified = classification.Train_sample_classification
        test_sample_classified  = classification.Test_sample_classification

        # add classfication into the list
        Classificatioin_result.append((classificaiton_tpye,year,seed,classified_img,test_sample_classified))

In [25]:
# convert classificaiton instances into a dataframe
Classification_instances_df =  pd.DataFrame(Classificatioin_result,
                                            columns=['classificaiton_tpye','year','seed','classified_img','test_sample_classified'])
Classification_instances_df

,classificaiton_tpye,year,seed,classified_img,test_sample_classified
0,Sentinel_Landsat,2017_2019,0,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
1,Sentinel_Landsat,2017_2019,1,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
2,Sentinel_Landsat,2017_2019,2,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
3,Sentinel_Landsat,2017_2019,3,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
4,Sentinel_Landsat,2017_2019,4,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
5,Sentinel_Landsat,2017_2019,5,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
6,Sentinel_Landsat,2017_2019,6,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
7,Sentinel_Landsat,2017_2019,7,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
8,Sentinel_Landsat,2017_2019,8,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
9,Sentinel_Landsat,2017_2019,9,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."


#### Add one of the classified img to map

In [33]:
one = Classification_instances_df.iloc[0][3]
one

In [34]:
Full_sample_classified_img = ee.ImageCollection("users/wangjinzhulala/North_China_Plain_Python/classification_img/Random_2017_2019")\
                               .sum()

In [35]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.addLayer(one,{'min':0,'max':1},'One')
Map.addLayer(Full_sample_classified_img,{'min':0,'max':1},'Full_sample_classified_img')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

#### calculate the accuracy of each radomness

In [30]:
acc = []

# calculate the accuracy of differnet randomness
for i in Classificatioin_result:
    
    accuracy = Accuracy_assesment(i[4]).Stp_1_Calculate_Accuracy()
    acc.append(accuracy)
    
    key = 'Recall_built'
    print(f'{i[1]}_{i[2]}-->Recall_built:{accuracy[key]}')

2017_2019_0-->Recall_built:94.62
2017_2019_1-->Recall_built:94.35
2017_2019_2-->Recall_built:94.98
2017_2019_3-->Recall_built:95.72
2017_2019_4-->Recall_built:94.73
2017_2019_5-->Recall_built:95.84
2017_2019_6-->Recall_built:94.72
2017_2019_7-->Recall_built:95.62
2017_2019_8-->Recall_built:95.5
2017_2019_9-->Recall_built:95.39
2014_2016_0-->Recall_built:94.7
2014_2016_1-->Recall_built:94.9
2014_2016_2-->Recall_built:94.59
2014_2016_3-->Recall_built:95.75
2014_2016_4-->Recall_built:94.24
2014_2016_5-->Recall_built:94.59
2014_2016_6-->Recall_built:94.51
2014_2016_7-->Recall_built:95.59
2014_2016_8-->Recall_built:95.76
2014_2016_9-->Recall_built:94.85


##### Export the classified_img

In [31]:
Classification_instances_df

,classificaiton_tpye,year,seed,classified_img,test_sample_classified
0,Sentinel_Landsat,2017_2019,0,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
1,Sentinel_Landsat,2017_2019,1,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
2,Sentinel_Landsat,2017_2019,2,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
3,Sentinel_Landsat,2017_2019,3,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
4,Sentinel_Landsat,2017_2019,4,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
5,Sentinel_Landsat,2017_2019,5,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
6,Sentinel_Landsat,2017_2019,6,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
7,Sentinel_Landsat,2017_2019,7,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
8,Sentinel_Landsat,2017_2019,8,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
9,Sentinel_Landsat,2017_2019,9,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."


In [32]:
for item in Classificatioin_result:
     
    Classification_type = item[0]
    year = item[1]
    seed = item[2]
    img  = item[3]
    
    
    export_name = f'{Classification_type}_{year}_{seed}'
    
#     # export to Google Storage
#     task = ee.batch.Export.image.toCloudStorage(image          = img,
#                                                 description    = export_name,
#                                                 bucket         = 'north_china_plain',
#                                                 fileNamePrefix = export_name,
#                                                 region         = North_China_Plain.geometry().bounds(),
#                                                 scale          = 30,
#                                                 maxPixels      = int(1e13),
#                                                 skipEmptyTiles = True)
    
    #export to GEE asset
    asset_path = f'users/wangjinzhulala/North_China_Plain_Python/classification_img/{Classification_type}_{year}'
    
    task = ee.batch.Export.image.toAsset(   image          = img,
                                            description    = export_name,
                                            assetId        = f'{asset_path}/{export_name}',
                                            region         = North_China_Plain.geometry().bounds(),
                                            scale          = 30,
                                            maxPixels      = int(1e13)) 
    
    task.start()
    
    print(f'Exporting {export_name} successful!')

Exporting Sentinel_Landsat_2017_2019_0 successful!
Exporting Sentinel_Landsat_2017_2019_1 successful!
Exporting Sentinel_Landsat_2017_2019_2 successful!
Exporting Sentinel_Landsat_2017_2019_3 successful!
Exporting Sentinel_Landsat_2017_2019_4 successful!
Exporting Sentinel_Landsat_2017_2019_5 successful!
Exporting Sentinel_Landsat_2017_2019_6 successful!
Exporting Sentinel_Landsat_2017_2019_7 successful!
Exporting Sentinel_Landsat_2017_2019_8 successful!
Exporting Sentinel_Landsat_2017_2019_9 successful!
Exporting Sentinel_Landsat_2014_2016_0 successful!
Exporting Sentinel_Landsat_2014_2016_1 successful!
Exporting Sentinel_Landsat_2014_2016_2 successful!
Exporting Sentinel_Landsat_2014_2016_3 successful!
Exporting Sentinel_Landsat_2014_2016_4 successful!
Exporting Sentinel_Landsat_2014_2016_5 successful!
Exporting Sentinel_Landsat_2014_2016_6 successful!
Exporting Sentinel_Landsat_2014_2016_7 successful!
Exporting Sentinel_Landsat_2014_2016_8 successful!
Exporting Sentinel_Landsat_2014